# TSIA202a - Second Practice Session : Spectral density estimation and periodogram
The goal of this second session is to provide a power spectral density estimator of a real, zero-mean, weakly stationary process $X_t$. We suppose that we have access to $n$ observations and we will use the FFT algorithm (that implements the DFT) using `numpy.fft.module`.
Recall (from the course) that the periodogram of the observations $X_0, \dots, X_{n-1}$ can be given as:
$$
I_n(\lambda) = \frac{1}{2\pi n}|\sum_{k=0}^{n-1} X_k e^{i\lambda k}|^2
$$

Moreover, the Hertglotz theorem provides a relation between the empirical autocovariance $\hat{\gamma}_n$ and the periodogram $I_n$:
$$
\hat{\gamma}_n(k) = \int_{0}^{2\pi}e^{i\lambda k}I_n({\lambda})d\lambda
$$

1. For a given $m \geq n$  we denote also the DFT as:
$$
DFT(X,m)(k) = \sum_{h=0}^{n-1}X_he^{-2i\pi\frac{kh}{m}}
$$
Show the following relation: 
$$I_n(\frac{2\pi k}{m}) = \frac{1}{2\pi n} |DFT(X,m)(k)|^2$$
2. provide a script that compute those $I_n(\frac{2\pi k}{m})$ for the time series mentioned in the first practice session
3. Show that $I_n(\lambda) = \frac{1}{2\pi} \sum_{k=0}^{n-1} \hat{\gamma}_n(k)e^{-i\lambda k}$
4. How to choose $m$ in order to get a simple relation between $\hat{\gamma}_n(k)$ and $I_n(\frac{2\pi k}{m})$ ? At the end, given a specific $\tilde{m}$ show that:
$$
\hat{\gamma}_n(k) = \frac{1}{n} IDFT\left(\left|DFT(X, \tilde{m})\right|^2, \tilde{m}\right)(k)
$$ Try this estimator on the autocovariance of previous time series of the first session.

5. In the case of white noise, estimate the variance of the periodogram for several values of $n$ and discuss about it.




**Billy Nicolas    Kaeppelin Matthieu**

## Question 1
We use the fact that the norm of a complex number is the same as it's conjugate
$$
I_n(\frac{2\pi k}{m}) = \frac{1}{2\pi n} =| \sum_{h=0}^{n-1} X_h \exp(\frac{2i\pi kh}{m}) |^2 = \frac{1}{2\pi n} | \overline{\sum_{h=0}^{n-1} X_h \exp(\frac{2i\pi kh}{m})}|^2 =\frac{1}{2\pi n} | \sum_{h=0}^{n-1} \overline{X_h} \exp(\frac{-2i\pi kh}{m}) |^2 
$$

$X_h$ is a real process, then we have: $\overline{X_h}=X_h$, hence:

$$
I_n(\frac{2\pi k}{m}) =  \frac{1}{2\pi n} |DFT(X,m)(k)|^2
$$



## Question 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
n = 1000
sigma = 1
a = 0.5
b= 2
K= 100
A0 = 2
lamb = 0.5
phi = np.random.uniform(0, 2*np.pi)

# Proccessus du TP1

def WN(sigma, n):
    # n est la taille du bruit que l'on génère
    # sigma est l'écart type du bruit

    return np.random.normal(0, sigma, n)

def process_ab(sigma,n,a,b):
    Z = WN(sigma,n)
    return a + b*Z + np.roll(Z,1)

def process_sum(sigma,n,a,K):
    Z = WN(sigma,n)

    X = a
    for e in range(0,K+1):
        X += (-1/2)**e * np.roll(Z,e) 
    
    return X

def process_harmonic(sigma,n,A,lamda):
    Z = WN(sigma,n)
    phi = np.random.uniform(0, 2*np.pi)
    return Z + A*np.cos(lamda*np.arange(n) + phi)

def I(signal, m):
    n = len(signal)
    I = np.zeros(m)
    dft = np.fft.fft(signal, m)
    
    for k in range(m):
        frequency = 2 * np.pi * k / m
        I[k] = (1 / (2 * np.pi * n)) * np.abs(dft[k])**2
    
    return I

## Question 3

$$
\frac{1}{2\pi}\sum_{k=0}^{n-1} \hat{\gamma _n}(k)e^{-i\lambda k} = \frac{1}{2\pi}\sum_{k=0}^{n-1}(\int_{0}^{2\pi} e^{i\mu k}I_n(\mu)\, d\mu )e^{-i\lambda k}\nonumber = \frac{1}{2\pi}\sum_{k=0}^{n-1}(\int_{0}^{2\pi} e^{i\mu k}\frac{1}{2\pi n} \left| \sum_{s=0}^{n-1} X_j e^{i\mu s} \right|^2 \, d\mu )e^{-i\lambda k}
$$

$$
= \frac{1}{2\pi}\sum_{k=0}^{n-1}(\int_{0}^{2\pi} e^{i\mu k}\frac{1}{2\pi n}\sum_{j=0}^{n-1}\sum_{p=0}^{n-1}X_j\overline{X_p}e^{i \mu j} e^{-i \mu p}   \, d\mu )e^{-i\lambda k} =\frac{1}{4\pi^2 n }\sum_{k=0}^{n-1}\sum_{j=0}^{n-1}\sum_{p=0}^{n-1}X_j\overline{X_p}e^{-ik\lambda}\int_{0}^{2\pi}e^{i\mu (k+j-p)}\, d\mu

$$

We know that $\int_{0}^{2\pi}e^{i\mu (k+j-p)}\, d\mu = 2 \pi$ when $(k+j-p) = 0$ and $0$ elsewhere, that simplifies one of the sum.

$$
\frac{1}{2\pi}\sum_{k=0}^{n-1} \hat{\gamma _n}(k)e^{-i\lambda k} = \frac{1}{2\pi n} \sum_{k=0}^{n-1}\sum_{j=-k}^{n-1-k} X_j \overline{X_{k+j}} e^{-ik\lambda}
$$

We make a changement of variable and we separate the exponential to get a produc of a complex sum and it's conjugate

$$
\frac{1}{2\pi}\sum_{k=0}^{n-1} \hat{\gamma _n}(k)e^{-i\lambda k} =  \frac{1}{2\pi n}|\sum_{k=0}^{n-1} X_k e^{i\lambda k}|^2 = I_n(\lambda)
$$

## Question 4

On utilise la question précédente:

$$ I_n(\frac{2\pi k}{m}) = \frac{1}{2\pi} \sum_{p=0}^{n-1} \hat{\gamma}_n(k)e^{-i \frac{2\pi k}{m} p} $$